In [27]:
import pandas as pd
import numpy as np
import os
from os.path import join, pardir
DATA_DIR = join(pardir, pardir, 'data', 'write_new_algorithm', 'gcp_cloud_storage')
DATA_DIR_ZK = join(pardir, pardir, 'data', 'write_zookeeper')

dfs = []
dfs2 = []
for mem in [512]:
    
    for size in [4, 65536, 262144]:
        df = pd.read_csv(join(DATA_DIR, f'persistent_{size}_{mem}_processed.csv'), index_col=0)
        df['size'] = size / 1024
        
        df['cpus'] = 0.33
        dfs.append(df)
        
for mem in [512]:
    
    for size in [4, 65536, 262144]:
        df = pd.read_csv(join(DATA_DIR, '1vCPU_512', f'persistent_{size}_1vCPU_{mem}_processed.csv'), index_col=0)
        df['size'] = size / 1024
        
        df['cpus'] = 1
        dfs2.append(df)
#print(dfs)
data = pd.concat([*dfs, *dfs2])


## Performance change

In [22]:
#print(data.groupby(['function', 'memory', 'size', 'cpus'])['time'].median())
#print(data.groupby(['function', 'memory', 'size', 'cpus'])['time'].mean())
#print(data.groupby(['function', 'memory', 'size', 'cpus'])['time'].quantile(.95))
#print(data.groupby(['function', 'memory', 'size', 'cpus'])['time'].quantile(.99))

small = data.loc[data['cpus'] == 0.33].groupby(['function', 'memory', 'size'])['time'].median()
large = data.loc[data['cpus'] == 1.00].groupby(['function', 'memory', 'size'])['time'].median()
print('Median', small/large)
small = data.loc[data['cpus'] == 0.33].groupby(['function', 'memory', 'size'])['time'].mean()
large = data.loc[data['cpus'] == 1.00].groupby(['function', 'memory', 'size'])['time'].mean()
print('Mean', small/large)

Median function     memory  size      
distributor  512     0.003906      0.948596
                     64.000000     0.985185
                     256.000000    1.036033
writer       512     0.003906      0.968738
                     64.000000     1.037377
                     256.000000    0.951149
Name: time, dtype: float64
Mean function     memory  size      
distributor  512     0.003906      0.981013
                     64.000000     0.956545
                     256.000000    1.023722
writer       512     0.003906      0.900045
                     64.000000     0.985159
                     256.000000    0.911737
Name: time, dtype: float64


### Cost reduction

In [28]:
import math
# PUT, COPY, POST, LIST requests (per 1,000 requests)


gcp_gbs = 0.0000025
gcp_ghzs = 0.0000100
gcp_invocations =  0.0000004

def func_cost(length, cpu, memory):
    return cpu * 2.4 * length * gcp_ghzs +  memory/1024 * length * gcp_gbs + gcp_invocations


In [29]:
import pandas as pd
vals= []
for idx, group in data.groupby(['function', 'memory', 'size', 'cpus']):
    
    #print(idx, group['time'].median())
    cost=func_cost(group['time'].median()/1000.0, idx[3], idx[1])
    #print('Cost', cost)
    vals.append([*idx, cost])
df = pd.DataFrame(vals, columns=['function', 'memory', 'size', 'cpus', 'cost'])

small = df.loc[df['cpus'] == 0.33].reset_index()
large = df.loc[df['cpus'] == 1.00].reset_index()
#print(small)
#print(large)
print('how much lower is cost', small['cost']/large['cost'])
print('cost reduction', 1-small['cost']/large['cost'])

how much lower is cost 0    0.378965
1    0.391701
2    0.407951
3    0.442999
4    0.460991
5    0.431448
Name: cost, dtype: float64
cost reduction 0    0.621035
1    0.608299
2    0.592049
3    0.557001
4    0.539009
5    0.568552
Name: cost, dtype: float64
